## Шаг 1. Изучение общей информации

In [ ]:
import pandas as pd
from pymystem3 import Mystem
m = Mystem()

In [2]:
data = pd.read_csv(r"C:\Users\User\yandex-praktikum-projects\research of bank borrowers\bank borrowers.csv")
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

- В столбце days_employed присутствуют отрицательные числа. И в пересчете на годы не во всех случаях результат похож на правду. Вероятно, при выгрузки данных произошел сбой. Данный столбец в исследовании не участвует, поэтому с этими данными никаких операций производить не буду.
- В столбце total_income есть отсутствующие данные. Необходимо их заполнить. Возможно, при заполнении анкеты потенциальные заемщики пропустили данный пункт.
- Столец total_income имеет неподходящий тип. Нужно исправить. 

## Шаг 2. Предобработка данных

### Обработка пропусков

In [4]:
data.isna().sum() # проверка количества пропущенных значений по столбцам

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [5]:
data['total_income'] = pd.to_numeric(data['total_income'], errors='coerce') # приведение типа данных к численному

In [6]:
data['education'] = data['education'].str.lower() # приведение столбца к нижнему регистру для исключения дублирующих значений


In [7]:
def age_group(age): # для заполнения отсутствующих значений столбца total_income необходимо рассчитать медианные значения
                    # дохода для каждой категории в зависимости от возраста и типа занятости        
    """
    Возвращает возврастную группу по значению возраста age, используя правила:
    - 'до 18 лет', если age <= 18 лет;
    - '19-27 лет', если age более 18 и менее 28, не включая 28;
    - '28-60 лет', если age более 28 и менее 60, включая 28 и 60
    - '61+ лет' — во всех остальных случаях.
    """
    
    if age <= 18:
        return 'до 18 дет'
    if age <= 27:
        return '19-27 лет'
    if age <= 60:
        return '28-60 лет'
    return '61+ лет' 

In [8]:
data['dob_years'].value_counts().sort_index(ascending=True) # выявлены значения возраста равные 0

0     101
19     14
20     51
21    111
22    183
23    254
24    264
25    357
26    408
27    493
28    503
29    545
30    540
31    560
32    510
33    581
34    603
35    617
36    555
37    537
38    598
39    573
40    609
41    607
42    597
43    513
44    547
45    497
46    475
47    480
48    538
49    508
50    514
51    448
52    484
53    459
54    479
55    443
56    487
57    460
58    461
59    444
60    377
61    355
62    352
63    269
64    265
65    194
66    183
67    167
68     99
69     85
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [9]:
# определение средних значений возраста по типу занятости
data_gr = data.pivot_table(index=['income_type'], columns='family_status', values='dob_years', aggfunc='mean')

def fill_null(family_status, income_type):
    return data_gr[family_status][income_type]

data['years_mean'] = data.apply(lambda row: fill_null(row['family_status'], row['income_type']), axis=1)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_mean
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40.350439
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,40.350439
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,40.350439
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,40.350439
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,58.653030


In [10]:
data['dob_years'] = data['dob_years'].replace(0, data['years_mean']) #замена нулевых значений посчитанными средними 


In [11]:
data['dob_years'].value_counts().sort_index(ascending=True) # данные заполнены

19.0     14
20.0     51
21.0    111
22.0    183
23.0    254
       ... 
71.0     58
72.0     33
73.0      8
74.0      6
75.0      1
Name: dob_years, Length: 73, dtype: int64

In [12]:
data['age_group'] = data['dob_years'].apply(age_group) # проведена категоризация, данные сохранены в отдельный столбец
display(data.head())  # проверка результата

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_mean,age_group
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40.350439,28-60 лет
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,40.350439,28-60 лет
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,40.350439,28-60 лет
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,40.350439,28-60 лет
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,58.653030,28-60 лет


In [13]:
# для удобства и более точного расчета значений, создана сводная таблица по категории возраста и типа занятости
pivot_table_for_total_income = data.pivot_table(index=['age_group','income_type'], columns='education', values='total_income', aggfunc='median')
display(pivot_table_for_total_income) #рассчитаны медианные значения total_income отдельно для типов занятости с разделением по уровню образования и возраста

education                         высшее      начальное  неоконченное высшее  \
age_group income_type                                                          
19-27 лет госслужащий      151445.859170            NaN        146237.440216   
          компаньон        169956.443471  151313.724737        159817.845686   
          пенсионер        214963.301941            NaN                  NaN   
          предприниматель  499163.144947            NaN                  NaN   
          сотрудник        144931.460486  153647.475906        136144.307247   
          студент           98201.625314            NaN                  NaN   
28-60 лет безработный      202722.511368            NaN                  NaN   
          в декрете                  NaN            NaN                  NaN   
          госслужащий      174770.798085  148339.290825        174064.740485   
          компаньон        209858.203315  136798.905143        191538.350577   
          пенсионер        143786.541900  110026.007572        111701.531885   
          сотрудник        168851.539875  123814.079468        163483.937551   
61+ лет   госслужащий      233952.871796            NaN                  NaN   
          компаньон        188400.822321            NaN        189610.009306   
          пенсионер        144240.768611   98095.380008        145557.484323   
          сотрудник        174933.507875  164412.796712        229339.200509   

education                        среднее  ученая степень  
age_group income_type                                     
19-27 лет госслужащий      133510.813687             NaN  
          компаньон        146634.999796             NaN  
          пенсионер         80044.196101             NaN  
          предприниматель            NaN             NaN  
          сотрудник        126184.739688             NaN  
          студент                    NaN             NaN  
28-60 лет безработный       59956.991984             NaN  
          в декрете         53829.130729             NaN  
          госслужащий      136542.894557   111392.231107  
          компаньон        160439.543585             NaN  
          пенсионер        118287.643240             NaN  
          сотрудник        137773.686974   198570.757322  
61+ лет   госслужащий      143847.253036             NaN  
          компаньон        173896.618274             NaN  
          пенсионер        111712.891757   177088.845999  
          сотрудник        133299.194693             NaN

In [14]:
# функция для заполнения отсутствующих значений total_income медианными значениями, полученными выше обработкой возможных ошибок
def fill_nan_total_income(age, income_type, education):
    try:
        return pivot_table_for_total_income[education][age][income_type]
    except:
        return 'ошибка'

In [15]:
# заполнения отсутствующих значений total_income медианными значениями в отдельный столбец
data['median_total_income'] = data.apply(lambda row: fill_nan_total_income(row['age_group'], row['income_type'],  row['education']), axis=1)
display(data.head())


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_mean,age_group,median_total_income
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40.350439,28-60 лет,168852
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,40.350439,28-60 лет,137774
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,40.350439,28-60 лет,137774
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,40.350439,28-60 лет,137774
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,58.653030,28-60 лет,118288


In [16]:
data['total_income'] = data['total_income'].fillna(data['median_total_income']) # подставляю медианные значения вместо пропущенных значений
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_mean,age_group,median_total_income
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,40.350439,28-60 лет,168852
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,40.350439,28-60 лет,137774
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,40.350439,28-60 лет,137774
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,40.350439,28-60 лет,137774
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,58.653030,28-60 лет,118288


In [17]:
data.isna().sum() # 1 ошибку при замене данных оставляю - это не сильно исказит итоговые выводы

children                  0
days_employed          2174
dob_years                 0
education                 0
education_id              0
family_status             0
family_status_id          0
gender                    0
income_type               0
debt                      0
total_income              1
purpose                   0
years_mean                0
age_group                 0
median_total_income       1
dtype: int64

**Вывод**

Для заполнения отсутствующих значений в столбце total_income необходимо рассчитать медианные значения с разбивкой по возрасту и типу занятости - это, на мой взгляд, решающие факторы, влияющие на уровень дохода. В пользу медианных значений отношу то, что крайние значения могут исказить среднее арифметическое значение            

### Замена типа данных

Замена тапа данных произошла на этапе 2 Предобработка данных методом to_numeric(). 

**Вывод**

В пользу этого метода отношу то, что если преобразование в числовой тип натыкается на ошибку, то есть возможность
аргументу errors передать значение, которое позволяет принудительно указывать нечисловые значения или просто игнорирует столбцы, содержащие эти значения.

### Обработка дубликатов

In [18]:
data.duplicated().sum() # проверка на наличие дубликатов

71

In [19]:
data = data.drop_duplicates().reset_index(drop=True) # удаление дубликатов с перезаписыванием индексов

In [20]:
data.duplicated().sum() # прверка результата

0

In [21]:
data['children'].unique() # проверка уникальных значений столбца children

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [22]:
data['children'] = data['children'].replace(-1, 1) # очевидно, здесь присутствует ошибка. Заменяю на положительное число
data['children'] = data['children'].replace(20, 2) # верояно, опечатка. Предполагаю, что 0 лишний

In [23]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [24]:
data['gender'].value_counts() # обнаружена ошибочное значение

F      14174
M       7279
XNA        1
Name: gender, dtype: int64

In [25]:
data = data[data['gender'] != 'XNA'] # так как не представляется возможным выяснить какого пола данный клиент
                                     # поэтому исключаю его из данных


**Вывод**

Выявлены дубликаты в данных. Удаление явных дубликатов произведено функцией drop_duplicates(). В столбце children методом replace() заменены отрицательные и ошибочные значения.

### Лемматизация

In [26]:
data['purpose'].sort_values(ascending=True).unique() # проверка уникальных значений в столбце

array(['автомобили', 'автомобиль', 'высшее образование',
       'дополнительное образование', 'жилье',
       'заняться высшим образованием', 'заняться образованием',
       'на покупку автомобиля', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля', 'на проведение свадьбы',
       'недвижимость', 'образование', 'операции с жильем',
       'операции с коммерческой недвижимостью',
       'операции с недвижимостью', 'операции со своей недвижимостью',
       'покупка жилой недвижимости', 'покупка жилья',
       'покупка жилья для сдачи', 'покупка жилья для семьи',
       'покупка коммерческой недвижимости', 'покупка недвижимости',
       'покупка своего жилья', 'получение высшего образования',
       'получение дополнительного образования', 'получение образования',
       'приобретение автомобиля', 'профильное образование',
       'ремонт жилью', 'свадьба', 'свой автомобиль',
       'сделка с автомобилем', 'сделка с подержанным автомобилем',
       'строительство 

In [27]:
def lemma_purpose(purpose):
    lemma = ' ' .join(m.lemmatize(purpose))
    return lemma

data['purpose_list'] = data['purpose'].apply(lemma_purpose)

In [28]:
def purpose_short(lemma): # можно выделить всего 4 цели, встречающиеся в разных формулировках
    if 'автомобиль' in lemma:
        return 'автомобиль'
    if 'образование' in lemma:
        return 'образование'
    if 'свадьба' in lemma:
        return 'свадьба'
    if 'недвижимость' or 'жилье' in lemma:
        return 'покупка/строительство недвижимости'
    

data['purpose_list'] = data['purpose_list'].apply(purpose_short)

display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_mean,age_group,median_total_income,purpose_list
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253876,покупка жилья,40.350439,28-60 лет,168852,покупка/строительство недвижимости
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,40.350439,28-60 лет,137774,автомобиль
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145886,покупка жилья,40.350439,28-60 лет,137774,покупка/строительство недвижимости
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267629,дополнительное образование,40.350439,28-60 лет,137774,образование
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,58.653030,28-60 лет,118288,свадьба


In [29]:
data['purpose_list'].unique()

array(['покупка/строительство недвижимости', 'автомобиль', 'образование',
       'свадьба'], dtype=object)

**Вывод**

В столбце purpose много значений сходых по смыслу, но отличающихся формулировкой. Для приведения их к единообразию необходима лемматизация. По ключевым словам выделены 4 цели и сохранены их в отдельном столбце.

### Категоризация данных

In [30]:
def income_group(row):
    '''категоризация в зависимости от дохода:
    'до 30 т.р.' - до 30 тыс. рублей включительно,
    '31-70 т.р.' - от 31 до 70 тыс. рублей включительно,
    '71-120 т.р.' - от 71 до 120 тыс. рублей включительно,
    '121-200 т.р.' - от 121 до 200 тыс. рублей включительно,
    'свыше 201 т.р.' - от 201 тыс. рублей.
   '''
    total_income = row['total_income']
    if total_income <= 30000:
        return 'до 30 т.р.'
    if total_income <= 70000:
        return '31-70 т.р.'
    if total_income <= 120000:
        return '71-120 т.р.'
    if total_income <= 200000:
        return '121-200 т.р.'
    return 'свыше 201 т.р.'

In [31]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21453 entries, 0 to 21453
Data columns (total 16 columns):
children               21453 non-null int64
days_employed          19350 non-null float64
dob_years              21453 non-null float64
education              21453 non-null object
education_id           21453 non-null int64
family_status          21453 non-null object
family_status_id       21453 non-null int64
gender                 21453 non-null object
income_type            21453 non-null object
debt                   21453 non-null int64
total_income           21452 non-null object
purpose                21453 non-null object
years_mean             21453 non-null float64
age_group              21453 non-null object
median_total_income    21452 non-null object
purpose_list           21453 non-null object
dtypes: float64(3), int64(4), object(9)
memory usage: 2.8+ MB
None


In [32]:
data['total_income'] = pd.to_numeric(data['total_income'], errors='coerce') # приведение типа данных к численному еще раз
data['income_group'] = data.apply(income_group, axis=1)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_mean,age_group,median_total_income,purpose_list,income_group
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,40.350439,28-60 лет,168852,покупка/строительство недвижимости,свыше 201 т.р.
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,40.350439,28-60 лет,137774,автомобиль,71-120 т.р.
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,40.350439,28-60 лет,137774,покупка/строительство недвижимости,121-200 т.р.
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,40.350439,28-60 лет,137774,образование,свыше 201 т.р.
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,58.653030,28-60 лет,118288,свадьба,121-200 т.р.


**Вывод**

Для удобства исследования необходимо поделить данные на категории в зависимости от уровня доходов. Без категоризации пришлось бы исследовать каждый доход, даже если бы от отличался от другого на 1 копейку.
Категории введены с учетом минимального, среднего, выше среднего, высокого доходов и очень высокого по статистике доходов.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [33]:
data_pivot_children = data.pivot_table(index=['debt'], columns='children', values='family_status_id', aggfunc='count')



In [34]:
data_pivot_children = data_pivot_children.fillna(0) # семьи с 5 детьми не имеют просрочек, 
                                                    # поэтому отсутствующее значение заменила на 0
display(data_pivot_children)

children,0,1,2,3,4,5
debt,,,,,,
0,13027.0,4410.0,1926.0,303.0,37.0,9.0
1,1063.0,445.0,202.0,27.0,4.0,0.0


In [35]:
# рассчитываю процент невозвращенных в срок кредитов ко всем выданным кредитам с учетом количества детей
chld0 = '{:2%}'.format(data_pivot_children[0][1] / data_pivot_children[0].sum())
chld1 = '{:2%}'.format(data_pivot_children[1][1] / data_pivot_children[1].sum())
chld2 = '{:2%}'.format(data_pivot_children[2][1] / data_pivot_children[2].sum())
chld3 = '{:2%}'.format(data_pivot_children[3][1] / data_pivot_children[3].sum())
chld4 = '{:2%}'.format(data_pivot_children[4][1] / data_pivot_children[4].sum())
chld5 = 0

In [36]:
columns_children = ['children', 0, 1, 2, 3, 4, 5]
persent_children = [['persent', chld0, chld1, chld2, chld3, chld4, chld5]]
persent_for_children = pd.DataFrame(data=persent_children, columns=columns_children) # составляю таблицу с результатом
display(persent_for_children)

,children,0,1,2,3,4,5
0,persent,7.544358%,9.165808%,9.492481%,8.181818%,9.756098%,0


In [37]:
df_example = data.groupby('children')['debt'].agg(['count', 'sum', 'mean', lambda x: 1 - x.mean()])
df_example.columns = ['Кол-во пользователей', 'Кол-во должников', '% должников', '% НЕдолжников']
df_example.style.format({'% должников': '{:.2%}', '% НЕдолжников': '{:.2%}'})

,Кол-во пользователей,Кол-во должников,% должников,% НЕдолжников
children,,,,
0,14090,1063,7.54%,92.46%
1,4855,445,9.17%,90.83%
2,2128,202,9.49%,90.51%
3,330,27,8.18%,91.82%
4,41,4,9.76%,90.24%
5,9,0,0.00%,100.00%


**Вывод**

Расчитаны процентные показатели кредитов с нарушением сроков возврата к общей сумме выданных кредитов в зависимости от количества детей.
- С увеличением количества детей падает общее количество выданных кредитов. 
- В целом, процент невозврата в срок увеличивается с увеличением количества детей в семьях. Выбиваются из общей статистики показатель в семьях с 3-мя детьми, возможно это связано с меньшим числом выданных кредитов. 
- Также интересно, что семьи с 5-ю детьми берут кредитов очень мало, возможно это связано с тем, что банки не одобряют кредитные заявки таким семьям.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [38]:
data_pivot_family_st = data.pivot_table(index=['debt'], columns='family_status_id', values = 'gender', aggfunc='count')
display(data_pivot_family_st)

family_status_id,0,1,2,3,4
debt,,,,,
0,11408,3762,896,1110,2536
1,931,388,63,85,274


In [39]:
# расчет значений невозвращенных кредитов в срок в зависимости от семейного положения
data_pivot_family_st = data_pivot_family_st.T
data_pivot_family_st['% return'] = round(data_pivot_family_st[1] / (data_pivot_family_st[0] + data_pivot_family_st[1]) * 100, 2)
display(data_pivot_family_st)

debt,0,1,% return
family_status_id,,,
0,11408,931,7.55
1,3762,388,9.35
2,896,63,6.57
3,1110,85,7.11
4,2536,274,9.75


**Вывод**

- Люди состоящие в гражданском браке  и неженатые/ незамужние более беспечные и не следят за сроком погашения кредитов. Процент непогашенных вовремя кредитов у этих категорий примерно одинаковый.
- Больше всего кредитов берут люди, состоящие в браке.
- Люди в разводе и вдовцы наиболее дисциплинированные - у них наименьшие показатели невыплаченных в срок кредитов.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [40]:
data_pivot_income = data.pivot_table(index=['debt'], columns='income_group', values='gender', aggfunc='count')
display(data_pivot_income)

income_group,121-200 т.р.,31-70 т.р.,71-120 т.р.,до 30 т.р.,свыше 201 т.р.
debt,,,,,
0,8249,1353,5219,20,4871
1,794,99,480,2,366


In [41]:
# расчет значений невозвращенных кредитов в срок в зависимости от уровня доходов
data_pivot_income = data_pivot_income.T
data_pivot_income['% return'] = round(data_pivot_income[1] / (data_pivot_income[0] + data_pivot_income[1]) * 100, 2)
display(data_pivot_income)

debt,0,1,% return
income_group,,,
121-200 т.р.,8249,794,8.78
31-70 т.р.,1353,99,6.82
71-120 т.р.,5219,480,8.42
до 30 т.р.,20,2,9.09
свыше 201 т.р.,4871,366,6.99


**Вывод**

- Люди с доходом до 30 тысяч рублей чаще невыплачивают в срок кредиты.
- Меньше просроченных кредитов имеею люди со средним доходом

- Как разные цели кредита влияют на его возврат в срок?

In [42]:
data_pivot_purpose = data.pivot_table(index=['debt'], columns='purpose_list', values='gender', aggfunc='count')
display(data_pivot_purpose)

purpose_list,автомобиль,образование,покупка/строительство недвижимости,свадьба
debt,,,,
0,3903,3643,10028,2138
1,403,370,782,186


In [43]:
data_pivot_purpose = data_pivot_purpose.T

In [44]:
# расчет значений невозвращенных кредитов в срок в зависимости от цели
data_pivot_purpose['% return'] = round(data_pivot_purpose[1] / (data_pivot_purpose[1] + data_pivot_purpose[0]) *100, 2)
display(data_pivot_purpose)

debt,0,1,% return
purpose_list,,,
автомобиль,3903,403,9.36
образование,3643,370,9.22
покупка/строительство недвижимости,10028,782,7.23
свадьба,2138,186,8.00


**Вывод**

- Больше всего кредитов берут на операции, связанные с недвижимостью. И чаще всего именно такие кредиты возвращают в срок. Видимо, это связано с долгосрочностью кредита.
- Самые невозвращаемые кредиты берут на покупку автомобиля и получения образования.

## Шаг 4. Общий вывод

 В  ходе исследования данные были изучены, приведены в требуемый вид:
 - Обнаружены и зполнены медианными значениями пропуски в столбце Total_income, также в нем изменен тип данных. Возможно, потенциальные клиенты не указали значения в заявках.
 - Также обнаружены незаполненные данные в столбце dob_years. Отсутствующие значения расчитаны исходя из среднего значения возраста с учетом типа занятости.
 - Приведены к одному регистру значения столбца education.
 - Обнаружены отрицательные значения в столбце days_employed. Вероятно, при загрузке данных возникла техническая ошибка.
 - Удалены явные дубликаты в данных. Вероятно, возникшие из-за внесения в базу некоторых заявок больше одного раза.
 - Исправлены ошибки в столбце children, возникшие, по всей видимости, из-за банальных опечаток.
 - Исключены из данных строка с ошибкой, возникшей при заполнении отсутствующих данных столбца с доходами, и строка с неизвестным значением столбца gender.
 - Проведени категоризации данных по возрасту и доходам для удобства и более наглядного анализа.
 - Столбец purpose содержал в себе множество идентичных целей, написанных разными формулировками. Что затрудняло процесс проверки гипотезы о зависимости цели кредита и его возврата в срок. С помощью лемматизации выделены 4 вида целей и применены к данным.
 - Удалось выяснить, что на возрат в срок кредитов влияет наличие детей: с увеличением количества детей уменьшается процент возвратов в срок и уменьшается количество выданных кредитов в целом. Вероятно, банки неохотно одобряют заявки таким семьям.
 - Семейное положение также имеет влияние на возврат кредитов в срок: люди состоящие в гразданском браке и незамужние/ неженатые чаще других не возвращают кредиты в срок. Больше всего кредитов берут женатые/замужние люди.
 - Уровень доходов также имеет влияние на невозрат кредитов в срок: клиенты с небольшим доходом чаще не возращают в срок.
 - Кредиты, взятые на покупку автомобиля и на образование, чаще не возвращают в срок. 
 На невозврат кредитов в срок влияют разные факторы, но в целом можно выделить идеальный образ потенциального заемщика: без детей, неженатый/незамужняя, с очень высоким или средним доходом, берущий кредит на покупку жилья.
 Вероятно, ошибки и пропуски в данных возникают по нескольким причинам: 
    - отсутствие строго контроля за заполнением некоторых пунктов анкет потенциальными клиентами; 
    - техническими сбоями  при выгрузке данных; 
    - внесением в базу одних и тех же анкет неоднократно.